In [7]:
from transformers import AutoTokenizer
from collections import defaultdict

In [8]:
# Corpus for testing
corpus = '''
    Object raspberrypi functools dict kwargs. Gevent raspberrypi functools. Dunder raspberrypi decorator dict didn't lambda zip import pyramid, she lambda iterate?
    Kwargs raspberrypi diversity unit object gevent. Import fall integration decorator unit django yield functools twisted. Dunder integration decorator he she future.
    Python raspberrypi community pypy. 
    Kwargs integration beautiful test reduce gil python closure. Gevent he integration generator fall test kwargs raise didn't visor he itertools...
    Reduce integration coroutine bdfl he python. Cython didn't integration while beautiful list python didn't nit!
    Object fall diversity 2to3 dunder script. Python fall for: integration exception dict kwargs dunder pycon. Import raspberrypi beautiful test import six web. Future 
    integration mercurial self script web. Return raspberrypi community test she stable.
    Django raspberrypi mercurial unit import yield raspberrypi visual rocksdahouse. Dunder raspberrypi mercurial list reduce class test scipy helmet zip?
'''

In [9]:
print(corpus)


    Object raspberrypi functools dict kwargs. Gevent raspberrypi functools. Dunder raspberrypi decorator dict didn't lambda zip import pyramid, she lambda iterate?
    Kwargs raspberrypi diversity unit object gevent. Import fall integration decorator unit django yield functools twisted. Dunder integration decorator he she future.
    Python raspberrypi community pypy. 
    Kwargs integration beautiful test reduce gil python closure. Gevent he integration generator fall test kwargs raise didn't visor he itertools...
    Reduce integration coroutine bdfl he python. Cython didn't integration while beautiful list python didn't nit!
    Object fall diversity 2to3 dunder script. Python fall for: integration exception dict kwargs dunder pycon. Import raspberrypi beautiful test import six web. Future 
    integration mercurial self script web. Return raspberrypi community test she stable.
    Django raspberrypi mercurial unit import yield raspberrypi visual rocksdahouse. Dunder raspberrypi me

## Normalisation

The normalization step involves some general cleanup, such as removing needless whitespace, lowercasing, and/or removing accents. If you’re familiar with Unicode normalization (such as NFC or NFKC), this is also something the tokenizer may apply.

The Transformers tokenizer has an attribute called backend_tokenizer that provides access to the underlying tokenizer from the Tokenizers library:

In [10]:
# import regex
import re

# convert to lower case
corpus = corpus.lower()
 
# remove numbers
corpus = re.sub(r'\d+','',corpus)
 
# remove all punctuation except words and space
corpus = re.sub(r'[^\w\s]','', corpus) 
 
# remove white spaces
corpus = corpus.strip()
corpus

'object raspberrypi functools dict kwargs gevent raspberrypi functools dunder raspberrypi decorator dict didnt lambda zip import pyramid she lambda iterate\n    kwargs raspberrypi diversity unit object gevent import fall integration decorator unit django yield functools twisted dunder integration decorator he she future\n    python raspberrypi community pypy \n    kwargs integration beautiful test reduce gil python closure gevent he integration generator fall test kwargs raise didnt visor he itertools\n    reduce integration coroutine bdfl he python cython didnt integration while beautiful list python didnt nit\n    object fall diversity to dunder script python fall for integration exception dict kwargs dunder pycon import raspberrypi beautiful test import six web future \n    integration mercurial self script web return raspberrypi community test she stable\n    django raspberrypi mercurial unit import yield raspberrypi visual rocksdahouse dunder raspberrypi mercurial list reduce clas

## Pre-tokenizer

A tokenizer cannot be trained on raw text alone. Instead, we first need to split the texts into small entities, like words. That’s where the pre-tokenization step comes in. A word-based tokenizer can simply split a raw text into words on whitespace and punctuation. Those words will be the boundaries of the subtokens the tokenizer can learn during its training.

In [12]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [13]:
word_freqs = defaultdict(int)
words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(corpus.strip())
new_words = [word for word, offset in words_with_offsets]
for word in new_words:
    word_freqs[word] += 1
print(word_freqs)

defaultdict(<class 'int'>, {'object': 3, 'raspberrypi': 10, 'functools': 3, 'dict': 3, 'kwargs': 5, 'gevent': 3, 'dunder': 5, 'decorator': 3, 'didnt': 4, 'lambda': 2, 'zip': 2, 'import': 5, 'pyramid': 1, 'she': 3, 'iterate': 1, 'diversity': 2, 'unit': 3, 'fall': 4, 'integration': 8, 'django': 2, 'yield': 2, 'twisted': 1, 'he': 4, 'future': 2, 'python': 5, 'community': 2, 'pypy': 1, 'beautiful': 3, 'test': 5, 'reduce': 3, 'gil': 1, 'closure': 1, 'generator': 1, 'raise': 1, 'visor': 1, 'itertools': 1, 'coroutine': 1, 'bdfl': 1, 'cython': 1, 'while': 1, 'list': 2, 'nit': 1, 'to': 1, 'script': 2, 'for': 1, 'exception': 1, 'pycon': 1, 'six': 1, 'web': 2, 'mercurial': 3, 'self': 1, 'return': 1, 'stable': 1, 'visual': 1, 'rocksdahouse': 1, 'class': 1, 'scipy': 1, 'helmet': 1})


## Model

Like BPE, WordPiece starts from a small vocabulary including the special tokens used by the model and the initial alphabet. Since it identifies subwords by adding a prefix (like ## for BERT), each word is initially split by adding that prefix to all the characters inside the word. So, the result look like below:

In [14]:
alphabet = []
for word in word_freqs.keys():
    if word[0] not in alphabet:
        alphabet.append(word[0])
    for letter in word[1:]:
        if f"##{letter}" not in alphabet:
            alphabet.append(f"##{letter}")

alphabet.sort()
alphabet

print(alphabet)

['##a', '##b', '##c', '##d', '##e', '##f', '##g', '##h', '##i', '##j', '##k', '##l', '##m', '##n', '##o', '##p', '##r', '##s', '##t', '##u', '##v', '##w', '##x', '##y', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y', 'z']


Thus, the initial alphabet contains all the characters present at the beginning of a word and the characters present inside a word preceded by the WordPiece prefix.

We also add the special tokens used by the model at the beginning of that vocabulary. In the case of BERT, it’s the list ["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"]:

In [15]:
vocab = ["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"] + alphabet.copy()
print(vocab)

['[PAD]', '[UNK]', '[CLS]', '[SEP]', '[MASK]', '##a', '##b', '##c', '##d', '##e', '##f', '##g', '##h', '##i', '##j', '##k', '##l', '##m', '##n', '##o', '##p', '##r', '##s', '##t', '##u', '##v', '##w', '##x', '##y', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y', 'z']


This list vocab will be our output at the end after adding all the commun pairs.

Next we need to split each word, with all the letters that are not the first prefixed by ##:

In [16]:
splits = {
    word: [c if i == 0 else f"##{c}" for i, c in enumerate(word)]
    for word in word_freqs.keys()
}

In [17]:
print(splits)

{'object': ['o', '##b', '##j', '##e', '##c', '##t'], 'raspberrypi': ['r', '##a', '##s', '##p', '##b', '##e', '##r', '##r', '##y', '##p', '##i'], 'functools': ['f', '##u', '##n', '##c', '##t', '##o', '##o', '##l', '##s'], 'dict': ['d', '##i', '##c', '##t'], 'kwargs': ['k', '##w', '##a', '##r', '##g', '##s'], 'gevent': ['g', '##e', '##v', '##e', '##n', '##t'], 'dunder': ['d', '##u', '##n', '##d', '##e', '##r'], 'decorator': ['d', '##e', '##c', '##o', '##r', '##a', '##t', '##o', '##r'], 'didnt': ['d', '##i', '##d', '##n', '##t'], 'lambda': ['l', '##a', '##m', '##b', '##d', '##a'], 'zip': ['z', '##i', '##p'], 'import': ['i', '##m', '##p', '##o', '##r', '##t'], 'pyramid': ['p', '##y', '##r', '##a', '##m', '##i', '##d'], 'she': ['s', '##h', '##e'], 'iterate': ['i', '##t', '##e', '##r', '##a', '##t', '##e'], 'diversity': ['d', '##i', '##v', '##e', '##r', '##s', '##i', '##t', '##y'], 'unit': ['u', '##n', '##i', '##t'], 'fall': ['f', '##a', '##l', '##l'], 'integration': ['i', '##n', '##t', '##e

Like BPE, WordPiece learns merge rules. The main difference is the way the pair to be merged is selected. Instead of selecting the most frequent pair, WordPiece computes a score for each pair, using the following formula:
$$
     score=(freq\_of\_pair)/(freq\_of\_first\_element × freq\_of\_second\_element)
$$
By dividing the frequency of the pair by the product of the frequencies of each of its parts, the algorithm prioritizes the merging of pairs where the individual parts are less frequent in the vocabulary.

Now that we are ready for training, let’s write a function that computes the score of each pair. We’ll need to use this at each step of the training:

In [18]:
#function that computes the scores for each pair of successive elements in each word
def compute_pair_scores(splits):
    letter_freqs = defaultdict(int)
    pair_freqs = defaultdict(int)
    for word, freq in word_freqs.items():
        split = splits[word]
        if len(split) == 1:
            letter_freqs[split[0]] += freq
            continue
        for i in range(len(split) - 1):
            pair = (split[i], split[i + 1])
            letter_freqs[split[i]] += freq
            pair_freqs[pair] += freq
        letter_freqs[split[-1]] += freq

    scores = {
        pair: freq / (letter_freqs[pair[0]] * letter_freqs[pair[1]])
        for pair, freq in pair_freqs.items()
    }
    return scores

In [19]:
#Example of score computing
pair_scores = compute_pair_scores(splits)
for i, key in enumerate(pair_scores.keys()):
    print(f"{key}: {pair_scores[key]}")
    if i >= 5:
        break

('o', '##b'): 0.05555555555555555
('##b', '##j'): 0.03333333333333333
('##j', '##e'): 0.007142857142857143
('##e', '##c'): 0.002976190476190476
('##c', '##t'): 0.004746835443037975
('r', '##a'): 0.01375


Now, finding the pair with the best score only takes a quick loop:

In [20]:
best_pair = ""
max_score = None
for pair, score in pair_scores.items():
    if max_score is None or max_score < score:
        best_pair = pair
        max_score = score

print(best_pair, max_score)

('e', '##x') 0.5


So the first merge to learn is ('e', '##x') -> 'ex', and we add 'ex' to the vocabulary dic that we've created before:

In [21]:
vocab.append("ex")

To continue, we need to apply that merge in our splits dictionary. Let’s write another function for this:

In [22]:
#function that apply the apply the previous merge in the splits dictionnary 
def merge_pair(a, b, splits):
    for word in word_freqs:
        split = splits[word]
        if len(split) == 1:
            continue
        i = 0
        while i < len(split) - 1:
            if split[i] == a and split[i + 1] == b:
                merge = a + b[2:] if b.startswith("##") else a + b
                split = split[:i] + [merge] + split[i + 2 :]
            else:
                i += 1
        splits[word] = split
    return splits

#### complexity of merge_pair
The complexity of the merge_pair function is O(N * M), where N is the number of words in the corpus and M is the maximum number of subword pieces (subtokens) in a single word.

Here's a breakdown of the complexity:

- The function iterates over each word in the corpus exactly once (O(N)).
- For each word, it searches through the subword pieces (subtokens) within that word to find and merge the pair (a, b).

In the worst-case scenario, where every word in the corpus contains the pair (a, b), the complexity is O(N * M), where N is the number of words and M is the maximum number of subtokens in a word.

In [23]:
#And we can have a look at the result of the first merge:
splits = merge_pair("e", "##x", splits)
splits["exception"]

['ex', '##c', '##e', '##p', '##t', '##i', '##o', '##n']

In [24]:
len(vocab)

52

Now we have everything we need to loop until we have learned all the merges we want. Let’s aim for a vocab size of 70:

In [25]:
vocab_size = 70
while len(vocab) < vocab_size:
    scores = compute_pair_scores(splits)
    best_pair, max_score = "", None
    for pair, score in scores.items():
        if max_score is None or max_score < score:
            best_pair = pair
            max_score = score
    splits = merge_pair(*best_pair, splits)
    new_token = (
        best_pair[0] + best_pair[1][2:] if best_pair[1].startswith("##")
        else best_pair[0] + best_pair[1]
    )
    vocab.append(new_token)

In [26]:
print(vocab)

['[PAD]', '[UNK]', '[CLS]', '[SEP]', '[MASK]', '##a', '##b', '##c', '##d', '##e', '##f', '##g', '##h', '##i', '##j', '##k', '##l', '##m', '##n', '##o', '##p', '##r', '##s', '##t', '##u', '##v', '##w', '##x', '##y', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'y', 'z', 'ex', 'kw', 'tw', 'ob', 'obj', 'dj', 'exc', '##ck', 'py', 'sh', 'wh', '##cks', '##cksd', 'im', 'imp', '##mm', '##mb', '##mbd', '##mmu']


To tokenize a new text, we pre-tokenize it, split it, then apply the tokenization algorithm on each word. That is, we look for the biggest subword starting at the beginning of the first word and split it, then we repeat the process on the second part, and so on for the rest of that word and the following words in the text:

In [27]:
def encode_word(word):
    tokens = []
    while len(word) > 0:
        i = len(word)
        while i > 0 and word[:i] not in vocab:
            i -= 1
        if i == 0:
            return ["[UNK]"]
        tokens.append(word[:i])
        word = word[i:]
        if len(word) > 0:
            word = f"##{word}"
    return tokens

In [33]:
print(encode_word("exception"))
print(encode_word("exercice"))
print(encode_word("python"))
print(encode_word("tokenize"))

['exc', '##e', '##p', '##t', '##i', '##o', '##n']
['ex', '##e', '##r', '##c', '##i', '##c', '##e']
['py', '##t', '##h', '##o', '##n']
['[UNK]']


Now, let’s write a function that tokenizes a text:

In [29]:
def tokenize(text):
    pre_tokenize_result = tokenizer._tokenizer.pre_tokenizer.pre_tokenize_str(text)
    pre_tokenized_text = [word for word, offset in pre_tokenize_result]
    encoded_words = [encode_word(word) for word in pre_tokenized_text]
    return sum(encoded_words, [])

In [31]:
tokenize("This is a simple demo of WordPiece tokenization using python")

['[UNK]',
 'i',
 '##s',
 '[UNK]',
 's',
 '##i',
 '##m',
 '##p',
 '##l',
 '##e',
 'd',
 '##e',
 '##m',
 '##o',
 'o',
 '##f',
 '[UNK]',
 '[UNK]',
 'u',
 '##s',
 '##i',
 '##n',
 '##g',
 'py',
 '##t',
 '##h',
 '##o',
 '##n']